# Computing covariate residuals
This workflow calculate the adjusted molecular phenotype data to adjust for additional covariates (i.e. kingship)

### Rational for covariate correction:
**FIXME:may need additional review for this paragarph:**


When we are doing xQtl analysis, typically, we performe variaous adjustment on both genotype matrix `X` as well as molecular phenotype matrix `Y`. This adjusting factors include sex, age, platform, etc. The overall goal of various adjustment is to rule out potential confounders in the analysis. 

However, when we are doing analysis, we may not always using the adjusted version of input data. Since `X` and `Y` are treated differerntly by differernt softwares, the corrected factors may not be identical. Specially, the correction in molecular phenotype matrix `Y` may be lesser that the genotype matrix `X`. This workflow implement additional covariate adjustment with regard to `Y`


### Method:

We regress out covariate `Z` out of `Y` by following methods: 

* Step 1: Regress `Y ~ Z`, get the estimated `βz`

* Step 2: Calculate `Y' = Y -  Z %*% βz` 

Since the output of step 2 is actually the residual of `Y` when regression `Z` out of `Y` , so our method is actually calculating the covariate residuals. 

**FIXME (need verfication):** Computationanlly, estimating `βz` needs `Z` to be full rank, result in `Z^T %*% Z` to be invertible. If `Z` is in low rank, we output an error.


### Input


### Output


(Other topics / Codes waiting to be adjust)

In [2]:
[global]
import os
# Work directory & output directory
parameter: wd = "./"
# The filename name for output data
# parameter: container = 'gaow/twas'
# name for the analysis output
parameter: name = 'ROSMAP'
# For cluster jobs, number commands to run per job
parameter: job_size = 1
# Wall clock time expected
parameter: walltime = "5h"
# Memory expected
parameter: mem = "16G"
# Number of threads
parameter: numThreads = 20

In [ ]:
[Residual_Expression_1]
# Path to the input molecular phenotype data.
parameter: molecular_pheno_whole = path
# Path to the factor file 
parameter: factor = path
input: molecular_pheno_whole,covariate
output: f'{wd}/cache/{name}.mol_phe.resid.bed'
task: trunk_workers = 1, trunk_size = 1, walltime = '4h',  mem = '20G', tags = f'{step_name}_{_output[0]:bn}'
R: expand = "$[ ]", stderr = f'{_output[0]}.stderr', stdout = f'{_output[0]}.stdout'#,volumes = [f'{wd:ad}:{genotype_list:ad}']
    library("dplyr")
    library("tibble")
    library("readr")
    library("modelr")
    library("purrr")
    factor = read_delim("$[_input[1]]",delim = "\t")
    pheno = read_delim("$[_input[0]]",delim = "\t")
    factor = factor%>%filter(str_detect(`#id`,"factor"))%>%select(-`#id`)%>%as.matrix()%>%t()
    pheno_id = pheno%>%select(gene_ID)
    pheno = pheno%>%select(rownames(factor))%>%as.matrix()%>%t()
    pheno_resid = .lm.fit(x = factor, y = pheno)$residuals
    pheno_output = cbind(pheno_id, pheno_resid%>%t())
    pheno_output%>%write_delim("$[_output[0]]",delim = "\t")